In [ ]:
#import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch
import torch.nn as nn
from torchvision import datasets,transforms
from transformers import ViTImageProcessor, ViTForImageClassification,get_linear_schedule_with_warmup

from PIL import Image
from torch.optim import AdamW
from torch.utils.data import random_split
warnings.filterwarnings('ignore')

In [ ]:
#linking my drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Data Preprocessing and loading
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])
dataset = datasets.ImageFolder('/content/drive/MyDrive/spices', transform=transform)

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224',do_rescale=False)

In [ ]:
#control panel
Batch_size=32#this states the size of each batch taken by the train
Epochs=7#number of iterations
Learning_rate=2e-5
dp_rate=0.4

In [ ]:
#Model class
class classifier(nn.Module):
    def __init__(self,n_classes,dp_rate):
        super(classifier,self).__init__()
        self.vit=ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
        self.dropout=nn.Dropout(dp_rate)
        # The output layer should have n_classes, not n_classes + 1
        self.out=nn.Linear(1000,n_classes)
        self.num_classes = n_classes
    def forward(self,inputs):
        pooled_output=self.vit(inputs).logits
        output=self.dropout(pooled_output)
        return self.out(output)

In [ ]:
#data preprocessing which is prior to capsuling it in a dataloader class
class Preprocessing:
  def __init__(self,data,transform,processor):
    self.data=data
    self.transform=transform
    self.processor=processor
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    image,label=self.data[idx]
    if not isinstance(image, torch.Tensor):
      if self.transform:
        image=self.transform(image)
    inputs=self.processor(images=image,return_tensors="pt")

    return inputs['pixel_values'].squeeze(0),label

In [ ]:
def Dataloader(data,transform,processor):
  dl=Preprocessing(data,transform,processor)
  return torch.utils.data.DataLoader(dl,batch_size=Batch_size,shuffle=True)

In [ ]:
#performing a train test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

In [ ]:
train_loader=Dataloader(train_data,transform,processor)
test_loader=Dataloader(test_data,transform,processor)

In [ ]:
#initializing GPU
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#model initialization
model=classifier(52,dp_rate)
model=model.to(device)

In [ ]:
#epochs,optim,scheduler
EPOCHS=Epochs
optimizer=AdamW(model.parameters(),lr=Learning_rate)
total_steps=len(train_loader)*EPOCHS
scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
loss_fn=nn.CrossEntropyLoss().to(device)

In [ ]:
def train(model, train_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model.train()
    total_loss = 0
    correct_predictions = 0
    num_batches = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs)
        _, preds = torch.max(output, dim=1)

        loss = loss_fn(output, labels)
        correct_predictions += (preds == labels).sum().item()
        num_batches += 1
        total_loss += loss.item()

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_loss = total_loss / num_batches
    accuracy = correct_predictions / n_examples * 100
    return avg_loss, accuracy

def test(model, test_loader, loss_fn, device, n_examples):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    num_batches = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            output = model(inputs)
            _, preds = torch.max(output, dim=1)

            loss = loss_fn(output, labels)
            correct_predictions += (preds == labels).sum().item()
            num_batches += 1
            total_loss += loss.item()

    avg_loss = total_loss / num_batches
    accuracy = correct_predictions / n_examples * 100
    return avg_loss, accuracy,preds

def train_model(model, train_loader, test_loader, loss_fn, optimizer, device, scheduler, n_examples_train, n_examples_test, num_epochs):
    for epoch in range(num_epochs):
        train_loss, train_acc = train(model, train_loader, loss_fn, optimizer, device, scheduler, len(train_data))
        test_loss, test_acc, _ = test(model, test_loader, loss_fn, device, len(test_data)) # Modified line: unpacking the 3rd return value into '_' (ignore)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%\n")


In [ ]:
train_model(model, train_loader, test_loader, loss_fn, optimizer, device, scheduler, len(train_data), len(test_data), EPOCHS)

In [ ]:
class_names=dataset.classes

In [ ]:
#chatgpt code(check it out later)
from PIL import Image
from torchvision import transforms
# Load the model architecture
model = classifier(52, dp_rate)  # Initialize your model
model = model.to(device)  # Move to device

# Load the model state dictionary (weights)
model.load_state_dict(torch.load('/content/drive/MyDrive/pr dataset.pt'))

# Now, you should be able to set the model to evaluation mode
model.eval()
# Load the trained model
# model.eval()  # Set the model to evaluation mode

def preprocess_image(image_path):
    """Loads and preprocesses a single image."""
    image = Image.open(image_path)
    image = transform(image)  # Apply the same transforms used during training
    inputs = processor(images=image, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0).to(device)

# Load and preprocess the image
image_path = '/content/drive/MyDrive/test/Spinach_test.jpeg'  # Replace with the actual path
input_tensor = preprocess_image(image_path)

# Make predictions
with torch.no_grad():
    output = model(input_tensor.unsqueeze(0))  # Add batch dimension
    predicted_class_index = torch.argmax(output).item()

# Interpret the prediction
# Assuming you have a list of class labels called 'class_names'
predicted_class_label = class_names[predicted_class_index]
print(f"Predicted class: {predicted_class_label}")

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/pr dataset.pt')

In [ ]:
model=torch.load('/content/drive/MyDrive/pr dataset.pt')